In [1]:
import spiketoolkit as st
import spikeextractors as se
import numpy as np
import spikemetrics.metrics as metrics
from spikemetrics.common import Epoch

22:24:14 [I] klustakwik KlustaKwik2 version 0.2.6
/Users/colehurwitz/miniconda3/envs/spikeinterface/lib/python3.7/site-packages/networkx-2.2-py3.7.egg/networkx/classes/reportviews.py:95: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Mapping, Set, Iterable


In [2]:
workshop_path = "/Users/colehurwitz/"
# workshop_path = "/afs/inf.ed.ac.uk/user/s17/s1727843/spiketutorials/Spike_sorting_workshop_2019/"
recording = se.MEArecRecordingExtractor(recording_path= workshop_path + "recordings_36cells_four-tetrodes_30.0_10.0uV_20-06-2019_14_48.h5")


Could not load plane information. Assuming probe is in yz plane


In [3]:
recording = se.load_probe_file(recording, workshop_path + 'tetrode_16.prb')

In [4]:
st.sorters.KlustaSorter.default_params()

{'probe_file': None,
 'adjacency_radius': None,
 'threshold_strong_std_factor': 5,
 'threshold_weak_std_factor': 2,
 'detect_sign': -1,
 'extract_s_before': 16,
 'extract_s_after': 32,
 'n_features_per_channel': 3,
 'pca_n_waveforms_max': 10000,
 'num_starting_clusters': 50}

In [5]:
sorting = st.sorters.run_klusta(recording,)

Klusta units are not unique! Using unique unit ids


In [6]:
num_spikes = 0
for unit_id in sorting.get_unit_ids():
    num_spikes += len(sorting.get_unit_spike_train(unit_id))
print(num_spikes)

5961


In [7]:
st.postprocessing.export_to_phy(recording, sorting, output_folder='phy_results')

In [8]:
renamed_unit_ids = []
for i in range(len(sorting.get_unit_ids())):
    renamed_unit_ids.append(i*2)
print(renamed_unit_ids)

[0, 2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 32, 34, 36, 38, 40, 42, 44, 46, 48, 50, 52, 54, 56]


In [9]:
subsorting = se.SubSortingExtractor(sorting, renamed_unit_ids=renamed_unit_ids)

In [10]:
metric_info = st.postprocessing.get_quality_metric_data(recording, subsorting, nPC=3, ms_before=1., ms_after=2., dtype=None, max_num_waveforms=np.inf, 
                                                        max_num_pca_waveforms=np.inf, save_waveforms=False, verbose=False)

spike_times = metric_info[0]
spike_clusters = metric_info[1]
channel_map = metric_info[2]
pc_features = metric_info[3]
pc_feature_ind = metric_info[4]

In [11]:
params = {
          "isi_threshold" : 0.0015,
          "min_isi" : 0.000166,
          "num_channels_to_compare" : 13,
          "max_spikes_for_unit" : 500,
          "max_spikes_for_nn" : 10000,
          "n_neighbors" : 4,
          'n_silhouette' : 10000,
          "quality_metrics_output_file" : "metrics.csv",
          "drift_metrics_interval_s" : 51,
          "drift_metrics_min_spikes_per_interval" : 10
         }

In [12]:
epochs_dict = {
    'red': (300000, 960000),
    'blue': (0, 300000)
}

In [13]:
epochs = []
for key in list(epochs_dict.keys()):
    epoch_name = key
    epoch_start_frame = epochs_dict[key][0]
    epoch_end_frame = epochs_dict[key][1]
    epochs.append(Epoch(name=key, start_time=recording.frame_to_time(epoch_start_frame), end_time=recording.frame_to_time(epoch_end_frame)))
epochs = sorted(epochs, key=lambda epoch: epoch.start_time)

In [14]:
metrics_pd = metrics.calculate_metrics(spike_times=spike_times, spike_clusters=spike_clusters, channel_map=channel_map, \
                                       pc_features=pc_features, pc_feature_ind=pc_feature_ind, params=params, epochs = None,
                                       cluster_ids=subsorting.get_unit_ids())

Calculating isi violations
 ▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒ 100% 
Calculating presence ratio
 ▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒ 100% 
Calculating firing rate
 ▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒ 100% 
Calculating PC-based metrics
['isolation_distance', 'l_ratio', 'd_prime', 'nearest_neighbor']
 ▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒░░░░░░░░░░░░░░░░░░░░░░░░░ 38% 

/Users/colehurwitz/miniconda3/envs/spikeinterface/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:877: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
/Users/colehurwitz/miniconda3/envs/spikeinterface/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:877: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
/Users/colehurwitz/miniconda3/envs/spikeinterface/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:1748: RuntimeWarning: invalid value encountered in greater_equal
  cond2 = (x >= self.b) & cond0


 ▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒ 100% 
Calculating silhouette score
 ▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒ 100% 
Calculating drift metrics
 ▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒ 100% 


/Users/colehurwitz/spiketoolkit/spikemetrics/metrics.py:387: RuntimeWarning: All-NaN slice encountered
  return np.nanmin(SS,0)


In [15]:
metrics_pd

,cluster_id,num_spikes,firing_rate,presence_ratio,isi_viol,isolation_distance,l_ratio,d_prime,nn_hit_rate,nn_miss_rate,silhouette_score,max_drift,cumulative_drift,epoch_name
0,0,73.0,2.434249,0.47,14.764692,9.555756e+00,0.150196,2.889648,0.388128,0.004359,NaN,0.0,0.0,complete_session
1,2,132.0,4.401655,0.71,0.000000,5.912475e+01,0.000627,5.883481,0.888889,0.003488,0.262411,0.0,0.0,complete_session
2,4,50.0,1.667294,0.40,0.000000,5.660483e+01,0.000364,4.098529,0.533333,0.001861,0.161531,0.0,0.0,complete_session
3,6,442.0,14.738876,0.99,0.000000,1.501192e+02,0.001854,6.344124,0.938914,0.006523,0.287030,0.0,0.0,complete_session
4,8,323.0,10.770717,0.96,0.000000,8.958531e+01,0.008489,5.157739,0.882353,0.008691,0.145944,0.0,0.0,complete_session
5,10,94.0,3.134512,0.56,0.000000,3.841046e+01,0.005086,4.455109,0.648936,0.004034,0.102197,0.0,0.0,complete_session
6,12,35.0,1.167106,0.29,0.000000,2.716322e+01,0.005666,1.587155,0.400000,0.002981,0.048543,0.0,0.0,complete_session
7,14,120.0,4.001505,0.73,0.000000,4.839145e+01,0.000606,3.928161,0.838889,0.004052,0.107961,0.0,0.0,complete_session
8,16,124.0,4.134888,0.68,0.000000,1.464779e+02,0.000193,8.342017,0.946237,0.002398,0.190781,0.0,0.0,complete_session
9,18,441.0,14.705530,0.97,0.000000,6.536383e+01,0.002485,4.503660,0.895692,0.009118,0.175234,0.0,0.0,complete_session


In [16]:
non_pc_metric_info = st.postprocessing.get_non_pc_quality_metric_data(recording, sorting)

spike_times = non_pc_metric_info[0]
spike_clusters = non_pc_metric_info[1]
channel_map = non_pc_metric_info[2]  
total_units = np.max(spike_clusters) + 1 

print(metrics.calculate_firing_rate(spike_times=spike_times, spike_clusters=spike_clusters, total_units=total_units))
print(metrics.calculate_isi_violations(spike_times=spike_times, spike_clusters=spike_clusters, total_units=total_units,
                                 isi_threshold=params['isi_threshold'], min_isi=params['min_isi']))
print(metrics.calculate_presence_ratio(spike_times=spike_times, spike_clusters=spike_clusters, total_units=total_units))

 ▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒ 100% 
(array([ 2.43424871,  4.40165521,  1.66729364, 14.73887576, 10.77071691,
        3.13451204,  1.16710555,  4.00150473,  4.13488822, 14.70552989,
        0.56687984,  6.33571583, 15.87263544,  2.06744411, 17.20647035,
        3.46797077,  2.73436157,  8.13639296,  9.73699485,  2.76770744,
        5.60210663,  4.60173044,  6.50244519,  1.60060189,  7.4694755 ,
       14.53880053, 12.97154451,  8.33646819,  7.1026709 ]), array([ 73., 132.,  50., 442., 323.,  94.,  35., 120., 124., 441.,  17.,
       190., 476.,  62., 516., 104.,  82., 244., 292.,  83., 168., 138.,
       195.,  48., 224., 436., 389., 250., 213.]))
 ▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒ 100% 
[14.76469247  0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.04221557  0.          5.01493878  0.56638754
  0.          0.          0.          0.          0.          9.75707418
  0.     

In [17]:
metric_info = st.postprocessing.get_quality_metric_data(recording, subsorting, nPC=3, ms_before=1., ms_after=2., dtype=None, max_num_waveforms=np.inf, 
                                                        max_num_pca_waveforms=np.inf, save_waveforms=False, verbose=False)

spike_times = metric_info[0]
spike_clusters = metric_info[1]
channel_map = metric_info[2]
pc_features = metric_info[3]
pc_feature_ind = metric_info[4]

spikes_for_nn = min(len(spike_times), params['max_spikes_for_nn'])
spikes_for_silhouette = min(len(spike_times), params['n_silhouette'])
epoch = Epoch('complete_session', 0, np.inf)
in_epoch = np.logical_and(spike_times > epoch.start_time, spike_times < epoch.end_time)

print(metrics.calculate_drift_metrics(spike_times, spike_clusters, total_units, pc_features, pc_feature_ind,
                                      params['drift_metrics_interval_s'], params['drift_metrics_min_spikes_per_interval']))
print(metrics.calculate_pc_metrics(spike_clusters,
                                   total_units,
                                   channel_map,
                                   pc_features,
                                   pc_feature_ind,
                                   params['num_channels_to_compare'],
                                   params['max_spikes_for_unit'],
                                   spikes_for_nn,
                                   params['n_neighbors']))
print(metrics.calculate_silhouette_score(spike_clusters,
                                         total_units,
                                         pc_features,
                                         pc_feature_ind,
                                         spikes_for_silhouette))
      

 ▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒ 100% 
(array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]), array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]))
['isolation_distance', 'l_ratio', 'd_prime', 'nearest_neighbor']
 ▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒░░░░░░░░░░░░░░░░░░░░░░░░░ 38% 

/Users/colehurwitz/miniconda3/envs/spikeinterface/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:877: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
/Users/colehurwitz/miniconda3/envs/spikeinterface/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:877: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
/Users/colehurwitz/miniconda3/envs/spikeinterface/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:1748: RuntimeWarning: invalid value encountered in greater_equal
  cond2 = (x >= self.b) & cond0


 ▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒ 100% 
(array([9.56174038e+00, 5.63256502e+01, 5.16924232e+01, 1.49823490e+02,
       8.92978162e+01, 3.44124137e+01, 2.59141478e+01, 4.85123687e+01,
       1.45933070e+02, 6.31448452e+01, 1.83715522e+14, 2.49934151e+01,
       1.22173057e+02, 1.06161643e+02, 7.95600174e+01, 1.63610864e+02,
       3.85832007e+01, 4.88897685e+01, 7.48647112e+01, 5.76895479e+02,
       4.68236422e+02, 3.17969881e+02, 5.28557508e+01, 1.98882159e+01,
       1.18541523e+02, 1.10522251e+02, 9.44298845e+01, 4.60706381e+01,
       4.01575880e+01]), array([1.72710703e-01, 7.92346000e-04, 6.71062438e-04, 1.85181181e-03,
       8.19537414e-03, 7.81368283e-03, 7.07173901e-03, 5.97595830e-04,
       2.05411286e-04, 2.37677583e-03,            nan, 1.20327598e-01,
       5.63054231e-04, 2.17282258e-04, 2.39460226e-03, 1.39765079e-04,
       1.85424041e-02, 1.01307566e-02, 1.15822030e-03, 1.92835582e-03,
       3.10861022e-05, 1.47445942e-04, 5.34237424e-03, 5.66080045e-02,
   

/Users/colehurwitz/spiketoolkit/spikemetrics/metrics.py:387: RuntimeWarning: All-NaN slice encountered
  return np.nanmin(SS,0)


In [30]:
metrics_iso = metrics.calculate_pc_metrics(spike_clusters,
                                   total_units,
                                   channel_map,
                                   pc_features,
                                   pc_feature_ind,
                                   params['num_channels_to_compare'],
                                   params['max_spikes_for_unit'],
                                   spikes_for_nn,
                                   params['n_neighbors'],
                                   metric_names=['isolation_distance', 'nearest_neighbor'])

['isolation_distance', 'nearest_neighbor']
 ▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒░░░░░░░░░░░░░░░░░░░░░░░░░ 38% 

/Users/colehurwitz/miniconda3/envs/spikeinterface/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:877: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
/Users/colehurwitz/miniconda3/envs/spikeinterface/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:877: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
/Users/colehurwitz/miniconda3/envs/spikeinterface/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:1748: RuntimeWarning: invalid value encountered in greater_equal
  cond2 = (x >= self.b) & cond0


 ▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒ 100% 


In [31]:
metrics_iso

(array([9.56174038e+00, 5.63256502e+01, 5.16924232e+01, 1.49823490e+02,
        8.92978162e+01, 3.44124137e+01, 2.59141478e+01, 4.85123687e+01,
        1.45933070e+02, 6.31448452e+01, 1.83715522e+14, 2.49934151e+01,
        1.22173057e+02, 1.06161643e+02, 7.70338075e+01, 1.63610864e+02,
        3.85832007e+01, 4.88897685e+01, 7.48647112e+01, 5.76895479e+02,
        4.68236422e+02, 3.17969881e+02, 5.28557508e+01, 1.98882159e+01,
        1.18541523e+02, 1.10522251e+02, 9.44298845e+01, 4.60706381e+01,
        4.01575880e+01]),
 array([1.72710703e-01, 7.92346000e-04, 6.71062438e-04, 1.85181181e-03,
        8.19537414e-03, 7.81368283e-03, 7.07173901e-03, 5.97595830e-04,
        2.05411286e-04, 2.37677583e-03,            nan, 1.20327598e-01,
        5.63054231e-04, 2.17282258e-04, 2.86995893e-03, 1.39765079e-04,
        1.85424041e-02, 1.01307566e-02, 1.15822030e-03, 1.92835582e-03,
        3.10861022e-05, 1.47445942e-04, 5.34237424e-03, 5.66080045e-02,
        9.73849350e-04, 2.43542035e-03

In [22]:
metrics.calculate_pc_metrics

<function spikemetrics.metrics.calculate_pc_metrics(spike_clusters, total_units, channel_map, pc_features, pc_feature_ind, num_channels_to_compare, max_spikes_for_cluster, spikes_for_nn, n_neighbors, metric_names=['isolation_distance', 'l_ratio', 'd_prime', 'nearest_neighbor'])>

In [23]:
|

SyntaxError: invalid syntax (<ipython-input-23-4b37ef281455>, line 1)